In [1]:
import pandas as pd
import numpy as np
import os,cv2
!pip install imutils
from imutils import paths

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Model, Input
from tensorflow.keras.layers import Dense, Dropout,Conv2D,Conv2DTranspose,Cropping2D,BatchNormalization,Flatten,Reshape,LeakyReLU,ReLU,ZeroPadding2D
from tensorflow.keras.optimizers import Adam
import functools
import tensorflow_addons as tfa
from matplotlib import pyplot as plt 
from datetime import timedelta
from tqdm.auto import tqdm
from kaggle_datasets import KaggleDatasets

try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Device:', tpu.master())
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.TPUStrategy(tpu)
except:
    strategy = tf.distribute.get_strategy()
print('Number of replicas:', strategy.num_replicas_in_sync)

  Created wheel for imutils: filename=imutils-0.5.4-py3-none-any.whl size=25860 sha256=374720de8cef33b8c6e68bec3ecb6c4f411b88c2ee90314e7136f9032d6d0d07
  Stored in directory: /root/.cache/pip/wheels/86/d7/0a/4923351ed1cec5d5e24c1eaf8905567b02a0343b24aa873df2
Successfully built imutils
Number of replicas: 1


In [ ]:

GCS_PATH1 = KaggleDatasets().get_gcs_path('flickr-image-dataset')

In [2]:
class config():
    Base_Dir = GCS_PATH1 + "flickr30k_images/"
    images_Dir = os.path.join(Base_Dir,"flickr30k_images")
    csv_file = os.path.join(Base_Dir,"results.csv")
    
    epochs=100
    Bs = 32 * strategy.num_replicas_in_sync
    Lr=1e-3
    

In [3]:
images_path = list(paths.list_images(config.images_Dir))

In [4]:
def load_images(imagePath):
    image = tf.io.read_file(imagePath)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.convert_image_dtype(image, dtype=tf.float32)
    image = tf.image.resize(image, (300, 300))
    return image

In [5]:
with strategy.scope():
    AUTOTUNE = tf.data.experimental.AUTOTUNE

    train_ds = tf.data.Dataset.from_tensor_slices(images_path)
    train_ds = train_ds.map(load_images,num_parallel_calls=AUTOTUNE).cache()
    train_ds = train_ds.repeat().shuffle(buffer_size=400,reshuffle_each_iteration=True)
    train_ds = train_ds.batch(config.Bs).prefetch(AUTOTUNE)


In [6]:
###DownSample Block
OUTPUT_CHANNELS = 3
latent_dim = 1024
n_filters = 64 

def downsample(filters, size, apply_instancenorm=True):
    initializer = tf.random_normal_initializer(0., 0.02)
    gamma_init = keras.initializers.RandomNormal(mean=0.0, stddev=0.02)

    result = keras.Sequential()
    result.add(Conv2D(filters, size, strides=2, padding='same',
                             kernel_initializer=initializer, use_bias=False))

    if apply_instancenorm:
        result.add(tfa.layers.InstanceNormalization(gamma_initializer=gamma_init))

    result.add(LeakyReLU())

    return result

###Upsample Block

def upsample(filters, size, apply_dropout=False):
    initializer = tf.random_normal_initializer(0., 0.02)
    gamma_init = keras.initializers.RandomNormal(mean=0.0, stddev=0.02)

    result = keras.Sequential()
    result.add(Conv2DTranspose(filters, size, strides=2,
                                      padding='same',
                                      kernel_initializer=initializer,
                                      use_bias=False))

    result.add(tfa.layers.InstanceNormalization(gamma_initializer=gamma_init))

    if apply_dropout:
        result.add(Dropout(0.5))

    result.add(ReLU())

    return result


'''Function to define a standard CNN model'''
def Encoder(InputShape,latent_dim):
    initializer = tf.random_normal_initializer(0., 0.02)
    gamma_init = keras.initializers.RandomNormal(mean=0.0, stddev=0.02)

    inp = Input(shape=InputShape, name='input_image')

    down1 = downsample(n_filters, 4, False)(inp) # (bs, 128, 128, 64)
    down2 = downsample(n_filters*2, 4)(down1) # (bs, 64, 64, 128)
    down3 = downsample(n_filters*4, 4)(down2) # (bs, 32, 32, 256)

    conv = Conv2D(n_filters*6, 4, strides=1,padding='same',
                         kernel_initializer=initializer,
                         use_bias=False)(down3) # (bs, 31, 31, 512)

    norm1 = tfa.layers.InstanceNormalization(gamma_initializer=gamma_init)(conv)

    leaky_relu = LeakyReLU()(norm1)

    
    flatten = Flatten()(leaky_relu)
    
    last = Dense(latent_dim,activation=None)(flatten)
    
    return Model(inputs=inp, outputs=last)


In [7]:
def Decoder():
    initializer = tf.random_normal_initializer(0., 0.02)
    gamma_init = keras.initializers.RandomNormal(mean=0.0, stddev=0.02)
    inp = Input(shape=latent_dim)
    x = Dense(units=37*37*6*n_filters,activation='relu')(inp) # 4x4 feature maps (with 6N occurances)
    x = Reshape(target_shape=(37, 37, 6*n_filters))(x)
    x = LeakyReLU()(x)
    x = Conv2DTranspose(n_filters*4, 4, strides=1,padding ='same' ,kernel_initializer=initializer,use_bias=False)(x)
    up1 = upsample(n_filters*2, 4, False)(x) 
    up2 = upsample(n_filters, 4)(up1)
    up2 = ZeroPadding2D()(up2)
    up3 = upsample(OUTPUT_CHANNELS, 4)(up2) 
    return Model(inputs=inp, outputs=up3)
   

In [ ]:
tf.keras.utils.plot_model(Decoder_fn, show_shapes=True, dpi=64)
tf.keras.utils.plot_model(Encoder_fn, show_shapes=True, dpi=64)

In [8]:
def vae_loss_function(x, x_recon, mu, logsigma, kl_weight=0.0005):
    latent_loss = 0.5 * tf.reduce_sum(tf.exp(logsigma) + tf.square(mu) - 1.0 - logsigma, axis=1)
    reconstruction_loss = tf.reduce_mean(tf.abs(x-x_recon), axis=(1,2,3))
    vae_loss = kl_weight * latent_loss + reconstruction_loss

    return vae_loss


In [16]:
##Model Creation

###Model Subclassing

class Vae_Class(keras.Model):
    def __init__(self,Encoder,Decoder,lambda_cycle=10):
        super(Vae_Class, self).__init__()
        self.dec = Decoder
        self.enc = Encoder
        self.lambda_cycle = lambda_cycle
        
    def compile(self,loss_function,opt):
        super(Vae_Class, self).compile()
        self.opt = opt
        self.loss = loss_function
    
    def sampling(self,z_mean, z_logsigma):
        try:
            batch, latent_dim = z_mean.shape
            epsilon = tf.random.normal(shape=(batch, latent_dim))
            print(epsilon)
            z = z_mean + tf.math.exp(0.5 * z_logsigma) * epsilon
            print(z)
        except:
            z=z_mean
        return z
        
    def train_step(self, batch_data):
        input_image = batch_data
        
        with tf.GradientTape(persistent=True) as tape:
            latent_space = self.enc(input_image)
            z_mean = latent_space[:, :latent_dim]
            print(z_mean.shape)
            z_logsigma = latent_space[:,latent_dim+1:]
            z = self.sampling(z_mean,z_logsigma ) # mean , log
            predicted = self.dec(z)
            
            loss = self.loss(input_image, predicted,z_mean,z_logsigma)
        grads = tape.gradient(loss, self.trainable_variables)

        # apply gradients to variables
        self.opt.apply_gradients(zip(grads, self.trainable_variables))
        
        return {
            "loss":loss
        }


with strategy.scope():
    opt= Adam(1e-3, beta_1=0.5)

In [10]:
with strategy.scope():
    encoder_fn = Encoder((300, 300, 3),latent_dim=2*latent_dim+1)
    decoder_fn = Decoder()

In [17]:
with strategy.scope():
    model = Vae_Class(encoder_fn,decoder_fn)

In [18]:
with strategy.scope():
    model.compile(vae_loss_function,opt)

In [19]:
history = model.fit(
        train_ds,
        epochs=1,verbose=True,steps_per_epoch=500,initial_epoch=0
    )

(None, 1024)


ResourceExhaustedError: in user code:

    /opt/conda/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:805 train_function  *
        return step_function(self, iterator)
    /opt/conda/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:795 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /opt/conda/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /opt/conda/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /opt/conda/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:3417 _call_for_each_replica
        return fn(*args, **kwargs)
    /opt/conda/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:788 run_step  **
        outpu